In [ ]:
import pandas as pd
import re
import json
import gc

In [121]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [122]:
with open("../data/bronze/Details.json", 'r') as json_file:
    data = json.load(json_file)

## FUNCTIONS

In [124]:
def get_number(text: str):
    cleaned = re.findall(r'\d+', text)[0]
    return int(cleaned)

In [125]:
def get_header(parent_container):
    information = {}
    try:
        card_header = parent_container.find_element(By.CLASS_NAME, "card-header")
        header_text = card_header.find_element(By.CLASS_NAME, "details").text
        header_date = card_header.find_element(By.CLASS_NAME, "date").text
        header_condition = parent_container.find_element(By.TAG_NAME, "strong").text
        
        information = {
            "Information": header_text,
            "Date": header_date,
            "Condition": header_condition
        }
    except Exception as e:
        print("Error when getting information details")
    finally:
        return information

In [126]:
def get_rating(parent_container):
    rating = {}
    try:        
        rating_container = parent_container.find_element(By.CLASS_NAME,"overall-rating")      
        rating_text = rating_container.find_element(By.TAG_NAME, "strong").text
        rating = {
            "Rating": rating_text
        }
    except Exception as e:
        print("Error when getting rating details")
    finally:
        return rating

In [127]:
def get_categories(parent_container):
    categories = {}
    try:        
        categories_container = parent_container.find_element(By.CLASS_NAME,"categories")
        section_tags = categories_container.find_elements(By.TAG_NAME, "section")
        for tag in section_tags:
            try:
                strong_text = tag.find_element(By.TAG_NAME, "strong").text
                
                categories_div = tag.find_element(By.CSS_SELECTOR, ".webmd-rate.on-mobile")
                categories_value = categories_div.get_attribute("aria-valuenow")

                categories[strong_text] = categories_value
            except Exception as e:
                continue
    except Exception as e:
        print("Error when getting categories details")
    finally:
        return categories

In [128]:
def get_comment(parent_container):
    descripton = {}
    try:
        desc_container = parent_container.find_element(By.CLASS_NAME,"description")
        desc_text = desc_container.find_element(By.CLASS_NAME, "description-text")

        descripton = {
            "Comment": desc_text.text
        }
    except Exception as e:
        print(f"Error when getting comments details")
    finally:
        return descripton

In [129]:
def get_reviews(url, driver, waitTime, lastCount):
    ctr = lastCount
    reviews_by_page = {}

    driver.get(url)
    
    container = waitTime.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".reviews-page .shared-reviews-container")))
    review_card_holder = container.find_elements(By.CSS_SELECTOR, ".review-details-holder")
    
    for holder in review_card_holder:
        
        review_details = holder.find_element(By.CSS_SELECTOR, ".review-details")
        
        headers = get_header(review_details)
        rating = get_rating(review_details)
        categories = get_categories(review_details)
        comment = get_comment(review_details)
          
        full_review = {**headers, **rating, **categories, **comment}
        
        reviews_by_page[f"Review {ctr}"] = full_review
        ctr += 1
        
    return reviews_by_page, ctr 

In [130]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
waitTime = WebDriverWait(driver,10)
reviews = {}
for supplement, information in data.items():
    supplement_review = {}
    numReviews = get_number(information["Reviews"]['NumberOfReviews'])
    url = information["Reviews"]['Url']

    if numReviews == 0:
        continue

    try:
        driver.get(url)       
        
        container = waitTime.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".reviews-page .shared-reviews-container")))
        ul_pagination = container.find_element(By.CSS_SELECTOR, ".pagination-holder .pagination-holder .pagination")
        li_items = ul_pagination.find_elements(By.TAG_NAME, 'li')

        pagination_urls = [item.find_element(By.TAG_NAME, 'a').get_attribute("href") for item in li_items]
        lastCount = 1
        
        for urls in pagination_urls:
            
            details = get_reviews(urls, driver, waitTime, lastCount)
            lastCount = details[1]
            supplement_review.update(details[0])
        #    tag_a = items.find_element(By.TAG_NAME, 'a')
        #    redirect_url = tag_a.get_attribute("href")

         #   details = get_reviews(redirect_url, driver, waitTime, lastCount)
        #    lastCount = details[1]
        #    supplement_review.update(details[0])
        # review_card_holder = container.find_elements(By.CSS_SELECTOR, ".review-details-holder")
        # details = {}
        # for i, holder in enumerate(review_card_holder, start=1):
            
        #     review_details = holder.find_element(By.CSS_SELECTOR, ".review-details")

        #     headers = get_header(review_details)
        #     rating = get_rating(review_details)
        #     categories = get_categories(review_details)
        #     comment = get_comment(review_details)
            
        #     full_review = {**headers, **rating, **categories, **comment}

        #     details[f"Review{i}"] = full_review

        reviews[supplement] = supplement_review



    except Exception as e:
        print(e)
        continue

driver.quit()
del driver                    # Remove from memory
gc.collect()                  # Force garbage collection
print("Resources cleaned up.")

Error when getting comments details
Error when getting comments details
Error when getting comments details
Error when getting comments details
Error when getting comments details
Error when getting comments details
Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=134.0.6998.179); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00C8C7F3+24435]
	(No symbol) [0x00C12074]
	(No symbol) [0x00AE06E3]
	(No symbol) [0x00B28B39]
	(No symbol) [0x00B28E8B]
	(No symbol) [0x00B1E1F1]
	(No symbol) [0x00B4D804]
	(No symbol) [0x00B1E114]
	(No symbol) [0x00B4DA34]
	(No symbol) [0x00B6F20A]
	(No symbol) [0x00B4D5B6]
	(No symbol) [0x00B1C54F]
	(No symbol) [0x00B1D894]
	GetHandleVerifier [0x00F970A3+3213347]
	GetHandleVerifier [0x00FAB0C9+3295305]
	GetHandleVerifier [0x00FA558C+3271948]
	GetHandleVerifier [0x00D27

KeyError: 'NumberOfReviews'

In [131]:
reviews

{'Acacia Rigidula': {'Review 1': {'Information': '',
   'Date': '7/28/2015',
   'Condition': 'Condition: Other',
   'Rating': 'Overall rating 2.0',
   'Effectiveness': '1',
   'Ease of Use': '2',
   'Satisfaction': '3',
   'Comment': 'Test July TR data'}},
 'Acai': {'Review 1': {'Information': 'Yael | Age: 45-54 | On supplement for less than 1 month |',
   'Date': '6/4/2024',
   'Condition': 'Condition: Other',
   'Rating': 'Overall rating 5.0',
   'Effectiveness': '5',
   'Ease of Use': '5',
   'Satisfaction': '5',
   'Comment': 'Having allergic reaction- my thung \\mouth \\airways got swallen. I took an allergy medication. I have used this acai powder from well known company on an empty stomach. one flat tsp. I have allergy only to solphites. I rearly get food allergies. I had allergy when made recipe with shreded cocnut and found out that the shreded coconuts all contains solphites..May be I am allergic to acai-never tried it before [not allergic to Aronia berry] or maybe to unrepor